In [3]:
import pandas as pd

# Membaca SentiSUM dataset
df = pd.read_excel("/kaggle/input/indonesia-summary-sentiment/SentiSUM.xlsx")
df

,category,news_text,manual_summary,Sentimen
0,tajuk utama,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...,neutral
1,teknologi,Selfie ialah salah satu tema terpanas di kalan...,Asus memperkenalkan ZenFone generasi keempat...,positive
2,hiburan,"Jakarta , CNN Indonesia - - Dinas Pariwisata P...",Dinas Pariwisata Provinsi Bengkulu kembali men...,positive
3,tajuk utama,Merdeka.com - Indonesia Corruption Watch ( ICW...,Indonesia Corruption Watch ( ICW ) meminta Kom...,neutral
4,tajuk utama,Merdeka.com - Presiden Joko Widodo ( Jokowi ) ...,Jokowi memimpin upacara penurunan bendera . Us...,positive
...,...,...,...,...
995,olahraga,"Rimanews - Yaya Toure dan agennya , Dimitri Se...","Yaya Toure dan agennya , Dimitri Seluk , berja...",positive
996,tajuk utama,"Jakarta , CNN Indonesia - - Aksi kejahatan sib...",Aksi kejahatan siber baru-baru ini menemukan a...,neutral
997,olahraga,AC Milan tampak tidak ingin menghentikan gelia...,AC Milan tampak tidak ingin menghentikan gelia...,positive
998,hiburan,"Jakarta , CNN Indonesia - - Di Indonesia , kun...",Kunyit ternyata dapat memutihkan gigi dengan c...,neutral


In [2]:
# Load BERT Tokenizer dari model peringkasan yang telah ada
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization")

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [5]:
import time
import torch
from tqdm import tqdm # Menggunakan tqdm untuk menampilkan progress bar

# Pastikan telah menginisialisasi tokenizer dan model sebelumnya

start_time = time.time()

# Gunakan CUDA jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device\t: ", device)

# Inisialisasi kolom baru 'auto_summary' dalam filtered_data
df['auto_summary'] = ""

for idx, row in tqdm(df.iterrows(), total=len(df)):
    ARTICLE_TO_SUMMARIZE = row['news_text']
    
    try:
        # generate summary
        input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE, return_tensors='pt', max_length=512, truncation=True).to(device)
        model.to(device)
        summary_ids = model.generate(input_ids,
                                     min_length=70,
                                     max_length=100,
                                     num_beams=10,
                                     repetition_penalty=2.5,
                                     length_penalty=1.0,
                                     early_stopping=True,
                                     no_repeat_ngram_size=2,
                                     use_cache=True,
                                     do_sample=True,
                                     temperature=0.8,
                                     top_k=50,
                                     top_p=0.95)

        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Simpan hasil ringkasan ke dalam dataset
        df.at[idx, 'auto_summary'] = summary_text

    except Exception as e:
        print(f"Error pada indeks {idx}: {e}")

print("Runtime\t: ", time.time() - start_time)

Device	:  cuda


100%|██████████| 1000/1000 [29:21<00:00,  1.76s/it]

Runtime	:  1761.1927225589752


In [9]:
# Menyimpan Dataset ke dalam file Excel
df.to_excel('Auto-SentiSUM.xlsx', index=False)
df

,category,news_text,manual_summary,Sentimen,auto_summary
0,tajuk utama,"Jakarta , CNN Indonesia - - Dokter Ryan Thamri...",Dokter Lula Kamal yang merupakan selebriti sek...,neutral,"ryan thamrin, dokter yang terkenal lewat acara..."
1,teknologi,Selfie ialah salah satu tema terpanas di kalan...,Asus memperkenalkan ZenFone generasi keempat...,positive,"meski telah tersedia banyak pilihan, asus zenf..."
2,hiburan,"Jakarta , CNN Indonesia - - Dinas Pariwisata P...",Dinas Pariwisata Provinsi Bengkulu kembali men...,positive,"jakarta, cnn indonesia - dinas pariwisata prov..."
3,tajuk utama,Merdeka.com - Indonesia Corruption Watch ( ICW...,Indonesia Corruption Watch ( ICW ) meminta Kom...,neutral,"peneliti icw, aradila caesar mengatakan moment..."
4,tajuk utama,Merdeka.com - Presiden Joko Widodo ( Jokowi ) ...,Jokowi memimpin upacara penurunan bendera . Us...,positive,usai prosesi penurunan bendera di halaman ista...
...,...,...,...,...,...
995,olahraga,"Rimanews - Yaya Toure dan agennya , Dimitri Se...","Yaya Toure dan agennya , Dimitri Seluk , berja...",positive,"yaya toure dan agennya, dimitri seluk, berjanj..."
996,tajuk utama,"Jakarta , CNN Indonesia - - Aksi kejahatan sib...",Aksi kejahatan siber baru-baru ini menemukan a...,neutral,"jakarta, cnn indonesia - aksi kejahatan siber ..."
997,olahraga,AC Milan tampak tidak ingin menghentikan gelia...,AC Milan tampak tidak ingin menghentikan gelia...,positive,ac milan tampak tidak ingin menghentikan gelia...
998,hiburan,"Jakarta , CNN Indonesia - - Di Indonesia , kun...",Kunyit ternyata dapat memutihkan gigi dengan c...,neutral,manfaat kunyit ternyata dapat mengurangi rasa ...
